In [1]:
spark

Out[ 1 ]: <pyspark.sql.session.SparkSession at 0x7f217bf43c90>

In [2]:
#Library Imports
from pyspark.sql import SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils
from functools import reduce  
from pyspark.sql import DataFrame

from pyspark.sql.types import StructType, StructField, DoubleType

#Schema Used
featureSchema = StructType([StructField('label', DoubleType(), True),
                     StructField('F1', DoubleType(), True),
                     StructField('F2', DoubleType(), True),
                     StructField('F3', DoubleType(), True),                  
                     StructField('F4', DoubleType(), True),       
                     StructField('F5', DoubleType(), True),       
                     StructField('F6', DoubleType(), True),       
                     StructField('F7', DoubleType(), True),       
                     StructField('F8', DoubleType(), True),       
                     StructField('F9', DoubleType(), True),       
                     StructField('F10', DoubleType(), True),       
                     StructField('F11', DoubleType(), True),                  
                     StructField('F12', DoubleType(), True),       
                     StructField('F13', DoubleType(), True),       
                     StructField('F14', DoubleType(), True),       
                     StructField('F15', DoubleType(), True),       
                     StructField('F16', DoubleType(), True),       
                     StructField('F17', DoubleType(), True),       
                     StructField('F18', DoubleType(), True)])   

#Loadind dataset and creating DataFrame
df = spark.read.csv('/FileStore/tables/SUSY.csv', header=True, schema=featureSchema)

#Creating feature vector
featuresUsed= ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol= "features")
assembled = assembler.transform(df)

#Splitting data into Training and Test datset for LSVM
(trainingData, testData) = assembled.randomSplit([0.7, 0.3])

#Training LSVM
lsvc = LinearSVC(maxIter=10, regParam=0.1)
pipeline1 = Pipeline(stages=[lsvc])
model1 = pipeline1.fit(trainingData)

#Generating predictions and evaluating model accuracy
predict1 =model1.transform(testData)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predict1)
print("Accuracy = %g" % (accuracy))

# Seperating TruePredictions and FalsePredictions generated by LSVM
predict1.createOrReplaceTempView("predictVIEW");

fp = spark.sql("select * from predictVIEW where prediction <> label")

tp = spark.sql("select * from predictVIEW where prediction = label")

fpDF = fp.drop('prediction', 'rawPrediction', 'probability')
(trainData, TestData) = fpDF.randomSplit([0.7, 0.3])

tpDF = tp.drop('prediction', 'rawPrediction', 'probability')
(TrainData, TEstData) = tpDF.randomSplit([0.7, 0.3])

#Training and evaluating GBT and DT in Phase 2
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
dt = DecisionTreeClassifier(labelCol= "label", featuresCol= "features", maxDepth=5, minInstancesPerNode=20, impurity= "gini")

pipeline2 = Pipeline(stages=[gbt])
model2 = pipeline2.fit(trainData)

pipeline3 = Pipeline(stages=[dt])
model3 = pipeline3.fit(TrainData)

predict2 =model2.transform(testData)

predict3 =model3.transform(TEstData)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predict2)
print("Accuracy = %g" % (accuracy))

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predict3)
print("Accuracy = %g" % (accuracy))

#Seperating TruePrections and FalsePredictions generated by GBT and DT
predict2.createOrReplaceTempView("predict2VIEW");

predict3.createOrReplaceTempView("predict3VIEW");

fp1 = spark.sql("select * from predict2VIEW where prediction <> label")

fp2 = spark.sql("select * from predict3VIEW where prediction <> label")

fp1DF = fp1.drop('prediction', 'rawPrediction', 'probability')
fp2DF = fp2.drop('prediction', 'rawPrediction', 'probability')

#Creating the combined DataFrame for Phase 3
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

finalDF = unionAll(tpDF, fp1DF, fp2DF)
finalDF.count()

#Training and Evalaution of Phase 3

(TData, TstData) = finalDF.randomSplit([0.7, 0.3])

layers = [18, 19, 20, 2]
trainer1 = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

pipeline4 = Pipeline(stages=[trainer1])
model4 = pipeline4.fit(TData)

predict4 =model4.transform(TstData)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predict4)
print("Accuracy = %g" % (accuracy))

predictionAndLabels = predict4.select("prediction", "label")
predictionAndLabels.rdd.take(2)

predictionAndLabels.rdd.map(tuple).take(2)
metrics = BinaryClassificationMetrics(predictionAndLabels.rdd.map(tuple))

print("Area under PR = %s" % metrics.areaUnderPR)

print("Area under ROC = %s" % metrics.areaUnderROC)

Accuracy = 0.749009
Accuracy = 0.328812
Accuracy = 0.964093
Accuracy = 0.988979
Area under PR = 0.969078970404
Area under ROC = 0.987863296233